***José Ángel Amaro Blanco***

# Multiple Testing

## Introducción

Conforme aumenta el número de pruebas de hipótesis que se realizan sobre conjuntos de datos, también aumenta la probabilidad de obtener resultados falsamente significativos debido al azar [1]. Para resolver esto se emplean distintas metodologías para reducir el número de falsos positivos que se obtienen de estas pruebas, y estas metodologías se conocen como correcciones de multiple testing.

En este trabajo considerará una base de datos con 83 muestras y 2308 variables de entradas, las cuales son la expresión génica estandarizada de diversos genes. La variable de salida es un valor numérico del 1 al 4, y corresponde a distintos tipos de cáncer. 

Se explorará el impacto que puede tener la medida en la que esté presente un gen en una muestra asociada a un tipo de cáncer. Se harán pruebas de hipótesis para ver si hay diferencias en las expresiones de genes de muestras pertenecientes a distintas clases de cáncer, y también se realizarán metodologías de corrección de multiple testing, analizando el impacto que tiene cada una de las metodologías en las pruebas de hipótesis para los diferentes grupos de datos.

## Preámbulo

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from scipy.stats import f_oneway

## Análisis exploratorio de datos

Observemos de forma preliminar el conjunto de datos con el que se va a trabajar:

In [2]:
df = pd.read_csv("khan.csv")
M = df.shape[1]; N = df.shape[0]

print(display(df.head()))
print(f"Numero de variables: {M} | Numero de observaciones: {N}")

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X2300,X2301,X2302,X2303,X2304,X2305,X2306,X2307,X2308,y
0,0.773344,-2.438405,-0.482562,-2.721135,-1.217058,0.827809,1.342604,0.057042,0.133569,0.565427,...,-0.027474,-1.660205,0.588231,-0.463624,-3.952845,-5.496768,-1.414282,-0.647600,-1.763172,2
1,-0.078178,-2.415754,0.412772,-2.825146,-0.626236,0.054488,1.429498,-0.120249,0.456792,0.159053,...,-0.246284,-0.836325,-0.571284,0.034788,-2.478130,-3.661264,-1.093923,-1.209320,-0.824395,2
2,-0.084469,-1.649739,-0.241308,-2.875286,-0.889405,-0.027474,1.159300,0.015676,0.191942,0.496585,...,0.024985,-1.059872,-0.403767,-0.678653,-2.939352,-2.736450,-1.965399,-0.805868,-1.139434,2
3,0.965614,-2.380547,0.625297,-1.741256,-0.845366,0.949687,1.093801,0.819736,-0.284620,0.994732,...,0.357115,-1.893128,0.255107,0.163309,-1.021929,-2.077843,-1.127629,0.331531,-2.179483,2
4,0.075664,-1.728785,0.852626,0.272695,-1.841370,0.327936,1.251219,0.771450,0.030917,0.278313,...,0.061753,-2.273998,-0.039365,0.368801,-2.566551,-1.675044,-1.082050,-0.965218,-1.836966,2


None
Numero de variables: 2309 | Numero de observaciones: 83


Corroboremos que no hay huecos:

In [3]:
df.isna().sum().unique()

array([0])

La instrucción `df.isna().sum()` devuelve el número de valores nulos por columna. Al aplicar `.unique()` vemos que el único valor devuelto es 0, lo que indica que no hay valores nulos en ninguna columna. Dado que 2309 columnas (2308 variables, más la variable de salida) *no tienen NaNs*, entonces podemos estar seguros de que no hay huecos en la base de datos. 

## Presencia de genes en distintos tipos de cáncer

Ahora comparemos la magnitud de la diferencia entre los promedios de presencia por gen para las observaciones de cáncer de clase 2 y clase 4.

In [4]:
"""
copilot me está guiando porque a veces la sintaxis de pandas
me revuelve, pero en esencia le dije como haría esto y le pedí que me guiara con ello.
"""
obsC2 = df[df['y'] == 2] # observaciones clase 2
obsC4 = df[df['y'] == 4] 

variable_diferencia = [] # diferencia de promedios

for columna in df.columns[:-1]: # en la ultima col está 'y'; no la quiero iterar
    promC2 = obsC2[columna].mean()
    promC4 = obsC4[columna].mean()
    diferencia = abs(promC2 - promC4)
    variable_diferencia.append((columna, diferencia))

"""
eventualmente le movi cosas para que fuera mas natural el codigo para mi:
ordenar en orden descendente de acuerdo con las diferencias
"""
diferenciasOrdenadas = sorted(variable_diferencia, reverse=True, key= lambda tupla: tupla[1])

In [5]:
for i in range(10):
    print(f"gen: {diferenciasOrdenadas[i][0]} | diferencia: {diferenciasOrdenadas[i][1]}")

gen: X187 | diferencia: 3.3231514359503445
gen: X509 | diferencia: 2.9065371735696557
gen: X2046 | diferencia: 2.424515064329655
gen: X2050 | diferencia: 2.4017826167103444
gen: X129 | diferencia: 2.165185075602759
gen: X1645 | diferencia: 2.0654596394648275
gen: X1319 | diferencia: 2.0459413337834484
gen: X1955 | diferencia: 2.037339977524138
gen: X1003 | diferencia: 2.011336784813793
gen: X246 | diferencia: 1.8378295007351724


Con esta información, si hacemos un análisis de inferencia, el modelo que obtengamos puede darle más peso al valor estandarizado de estos genes que al de los demás como criterio de decisión al clasificar entre $y = 2$ o $y = 4$. Después de todo, si en promedio no hay diferencia entre la presencia de un gen para una observacion clase 2 con una de clase 4, entonces realmente éste no es decisivo, o no influye; *no tiene **peso***

## Multiple testing

Ahora hagamos pruebas t para la hipótesis nula de que *las muestras de los genes que pertenecen al cáncer de clase 2 son idénticas a los de clase 4*, y obtengamos los p-values correspondientes de cada prueba

In [6]:
### con ayuda de copilot
pValues = []
for col in df.columns[:-1]:
    tStat, pVal = ttest_ind(obsC2[col], obsC4[col], equal_var=False)
    pValues.append(pVal) 

Podríamos sentarnos a analizar los resultados, pero recordemos que para $m$ comparaciones independientes, el Family-Wise Error Rate (FWER), es decir, la probabilidad de cometer al menos un error de tipo I, es dada por la siguiente ecuación: 

$FWER = 1 - (1 - \alpha)^m$

Considerando que estamos haciendo 2308 (``M`` - 1) pruebas de hipótesis, el FWER para una significancia $\alpha$ de 0.05 es: 

$FWER = 1 - (1 - 0.05)^{2308} \rarr FWER = 100\%$  

Es decir, podemos estar prácticamente seguros de que se cometerá al menos un error de tipo 1 durante estas comparaciones. Así que ahora probemos usando métodos que corrigen p-values disminuyendo tanto el FWER como el FDR [1]:

- FWER: 
    - Bonferroni: El método más estricto de control de falsos positivos. Útil para cuando se hacen pocas pruebas de hipótesis
    - Holm-Bonferroni: Es menos estricto que la corrección Bonferroni, y útil para un número más moderado de hipótesis
- FDR:
    - Benjamini-Hochberg: La mejor opción para manejar un número grande de hipótesis

Estas metodologías son útiles para asegurarse que hallazgos que se hagan con pruebas de hipótesis sean estadísticamente significativos, y se usan mucho en el ámbito de la genómica. Además, en el ámbito de Machine Learning, estos métodos pueden ser útiles como forma de reducir el número de variables de un modelo. 

In [7]:
### sintaxis con ayuda de copilot
rejectBonf, pValsBonf, _, _ = multipletests(pValues, alpha=0.05, method='bonferroni') # el más estricto
rejectHolm, pValsHolm, _, _ = multipletests(pValues, alpha=0.05, method='holm') # deberíamos ver más genes significativos
rejectFDRBH, pValsFDRBH, _, _ = multipletests(pValues, alpha=0.05, method='fdr_bh') 

genes = df.columns[:-1]

genesSignificativosBonferroni = genes[rejectBonf]
genesSignificativosHolm = genes[rejectHolm]
genesSignificativosBenjaminiHochberg = genes[rejectFDRBH]

# Imprimir resultados
print("\nGenes significativos con Bonferroni:", genesSignificativosBonferroni, 
      f"\nNúmero de genes significativamente diferentes: {len(genesSignificativosBonferroni)}")
print("\nGenes significativos con Holm-Bonferroni:", genesSignificativosHolm,
      f"\nNúmero de genes significativamente diferentes: {len(genesSignificativosHolm)}")
print("\nGenes significativos con Benjamini-Hochberg:", genesSignificativosBenjaminiHochberg,
      f"\nNúmero de genes significativamente diferentes: {len(genesSignificativosBenjaminiHochberg)}")


Genes significativos con Bonferroni: Index(['X2', 'X36', 'X67', 'X129', 'X174', 'X187', 'X188', 'X229', 'X246',
       'X251', 'X338', 'X348', 'X368', 'X372', 'X373', 'X380', 'X430', 'X433',
       'X509', 'X545', 'X554', 'X558', 'X566', 'X603', 'X655', 'X714', 'X762',
       'X910', 'X951', 'X971', 'X1003', 'X1021', 'X1023', 'X1055', 'X1070',
       'X1093', 'X1105', 'X1110', 'X1112', 'X1132', 'X1194', 'X1196', 'X1207',
       'X1217', 'X1298', 'X1319', 'X1327', 'X1330', 'X1372', 'X1389', 'X1416',
       'X1610', 'X1626', 'X1634', 'X1645', 'X1706', 'X1708', 'X1723', 'X1738',
       'X1799', 'X1888', 'X1896', 'X1911', 'X1924', 'X1954', 'X1955', 'X1980',
       'X2046', 'X2050', 'X2115', 'X2146', 'X2247'],
      dtype='object') 
Número de genes significativamente diferentes: 72

Genes significativos con Holm-Bonferroni: Index(['X2', 'X36', 'X67', 'X129', 'X174', 'X187', 'X188', 'X229', 'X246',
       'X251', 'X338', 'X348', 'X368', 'X372', 'X373', 'X380', 'X430', 'X433',
       'X509',

In [8]:
print(f"Numero de genes significativamente distintos sin correccion: {np.sum(np.array(pValues) < 0.05)}")

Numero de genes significativamente distintos sin correccion: 547


A pesar de que uno esperaría ver más genes significativos en la metodología de Holm-Bonferroni que con la corrección Bonferroni, encontramos que, en este caso, los resultados de ambas fueron los mismos. Mismo número de genes significativamente diferentes, y mismos genes. En cambio, encontramos que con la metodología de Benjamini-Hochberg sí se encontraron más genes significativamente diferentes.

Aun así, con estas tres metodologías no solo logramos mitigar falsos positivos por hacer múltiples pruebas, sino que también redujimos el número de variables explicativas desde aproximadamente un $87.18\%$ hasta un $96.88\%$. Además, considerando 547 genes significativamente distintos sin corrección, si no se hicieran correcciones se cometerían errores tipo 1 con mínimo 251 - 475 genes.

Ahora hagamos pruebas ANOVA para comparar las medias de cada gen entre las cuatro clases de cáncer, y obtengamos los p-values correspondientes.

In [9]:
# observaciones de otras clases para el anova
obsC1 = df[df['y'] == 1]
obsC3 = df[df['y'] == 3]

pValuesANOVA = []

# ANOVA entre todos los registros por clase
for col in df.columns[:-1]:
    fstat, fpval = f_oneway(obsC1[col], obsC2[col], obsC3[col], obsC4[col])
    pValuesANOVA.append(fpval)

Ahora hagamos correcciones similares a las realizadas con las observaciones asociadas a cáncer de clase 2 y 4, para todos los genes

In [10]:
# se aplican correcciones
rejectBonf, pValsBonf, _, _ = multipletests(pValuesANOVA, alpha=0.05, method='bonferroni') # el más estricto
rejectHolm, pValsHolm, _, _ = multipletests(pValuesANOVA, alpha=0.05, method='holm') # deberíamos ver más genes significativos
rejectFDRBH, pValsFDRBH, _, _ = multipletests(pValuesANOVA, alpha=0.05, method='fdr_bh') 

genes = df.columns[:-1]

# genes significativos
genesSignificativosBonferroni = genes[rejectBonf]
genesSignificativosHolm = genes[rejectHolm]
genesSignificativosBenjaminiHochberg = genes[rejectFDRBH]

# Imprimir resultados
print("\nGenes significativos con Bonferroni:", genesSignificativosBonferroni, 
      f"\nNúmero de genes significativamente diferentes: {len(genesSignificativosBonferroni)}")
print("\nGenes significativos con Holm-Bonferroni:", genesSignificativosHolm,
      f"\nNúmero de genes significativamente diferentes: {len(genesSignificativosHolm)}")
print("\nGenes significativos con Benjamini-Hochberg:", genesSignificativosBenjaminiHochberg,
      f"\nNúmero de genes significativamente diferentes: {len(genesSignificativosBenjaminiHochberg)}")


Genes significativos con Bonferroni: Index(['X1', 'X2', 'X3', 'X17', 'X29', 'X33', 'X36', 'X50', 'X52', 'X54',
       ...
       'X2273', 'X2275', 'X2276', 'X2279', 'X2289', 'X2294', 'X2299', 'X2301',
       'X2303', 'X2304'],
      dtype='object', length=404) 
Número de genes significativamente diferentes: 404

Genes significativos con Holm-Bonferroni: Index(['X1', 'X2', 'X3', 'X17', 'X29', 'X33', 'X36', 'X50', 'X52', 'X54',
       ...
       'X2273', 'X2275', 'X2276', 'X2279', 'X2289', 'X2294', 'X2299', 'X2301',
       'X2303', 'X2304'],
      dtype='object', length=412) 
Número de genes significativamente diferentes: 412

Genes significativos con Benjamini-Hochberg: Index(['X1', 'X2', 'X3', 'X9', 'X12', 'X17', 'X21', 'X22', 'X27', 'X29',
       ...
       'X2289', 'X2294', 'X2295', 'X2297', 'X2299', 'X2300', 'X2301', 'X2302',
       'X2303', 'X2304'],
      dtype='object', length=1162) 
Número de genes significativamente diferentes: 1162


In [11]:
print(f"Numero de genes significativamente distintos sin correccion: {np.sum(np.array(pValuesANOVA) < 0.05)}")

Numero de genes significativamente distintos sin correccion: 1303


Con estas tres metodologías no solo logramos mitigar falsos positivos por hacer múltiples pruebas, sino que también redujimos el número de variables explicativas desde aproximadamente un $49.69\%$ hasta un $82.5\%$. Además, considerando 1303 genes significativamente distintos sin corrección, si no se hicieran correcciones se cometerían errores tipo 1 con mínimo 141 - 899 genes.

## Conclusiones

En conclusión, se pudo comprobar el efecto que tienen 3 de las distintas correcciones de multiple testing que existen, y se observó la relevancia que tuvieron para reducir la cantidad de genes con expresiones significativamente diferentes; las correcciones del FWER fueron mucho más estrictas que el procedimiento de Benjamini-Hochberg para el FDR.

Como trabajo futuro queda usar estas metodologías para reducir la dimensionalidad de un conjunto de datos usado para entrenar un modelo con aprendizaje automático. Cabe destacar que para evitar fuga de datos se tendrían que hacer las correcciones solamente con datos de entrenamiento, y para eso sería mejor trabajar con un conjunto de datos con más observaciones (en este caso solo contábamos con 83 observaciones).

## Referencias

[1] Statsig. (2025, 25 febrero). *Holm-Bonferroni correction: controlling false positives*. Statsig. Recuperado 17 de diciembre de 2025, de https://www.statsig.com/perspectives/holm-bonferroni-correction-false-positives

## Bibliografía

- Biostatistics. (2020, 27 octubre). Mini lecture: Multiple testing [Vídeo]. YouTube. https://www.youtube.com/watch?v=RUX94txw4Qo
- Wikipedia contributors. (s. f.). Multiple comparisons problem. Wikipedia. https://en.wikipedia.org/wiki/Multiple_comparisons_problem
- StatQuest with Josh Starmer. (2017, 10 enero). False Discovery Rates, FDR, clearly explained [Vídeo]. YouTube. https://www.youtube.com/watch?v=K8LQSvtjcEo
- Silicon Genetics. (2003). Multiple Testing Corrections. Recuperado 17 de diciembre de 2025, de https://physiology.med.cornell.edu/people/banfelder/qbio/resources_2008/1.5_GenespringMTC.pdf